# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The default looks reasonable, although I have some doubts as to whether it makes sense to discard the data around the edge in this case. 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [4]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

import nltk
nltk.download('punkt')

## TODO #1: Select appropriate values for the Python variables below.
batch_size=1
#batch_size=10
#batch_size = 64          # batch size, we have 10GB of GPU memory, let's use it
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = False    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, max_batch_size=batch_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) + list(encoder.bn.parameters())

# TODO #4: Define the optimizer.
#optimizer = torch.optim.SGD(params, lr=0.01)

# this data is probably pretty sparse, and defaults are probably ok
#http://ruder.io/optimizing-gradient-descent/
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /home/sthenc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.62s)
creating index...


  0%|          | 897/414113 [00:00<00:46, 8966.76it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:42<00:00, 9632.66it/s] 


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# temporary
%load_ext autoreload
%autoreload 2

from model import EncoderCNN, DecoderRNN

# Open the training log file.
f = open(log_file, 'w')

## running the training locally
#old_time = time.time()
#response = requests.request("GET", 
#                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                            headers={"Metadata-Flavor":"Google"})


for epoch in range(1, num_epochs+1):

    for i_step in range(1, total_step+1):
        
        ## running the training locally
        #if time.time() - old_time > 60:
        #    old_time = time.time()
        #    requests.request("POST", 
        #                     "https://nebula.udacity.com/api/v1/remote/keep-alive", 
        #                     headers={'Authorization': "STAR " + response.text})

        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)

        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()

        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)

        # Calculate the batch loss.
        loss = criterion(outputs.contiguous().view(-1, vocab_size), captions.contiguous().view(-1))

        # Backward pass.
        loss.backward(retain_graph=True)

        # Update the parameters in the optimizer.
        optimizer.step()

        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))

        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()

        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()

        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)

    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch [1/3], Step [100/414113], Loss: 4.3513, Perplexity: 77.57642
Epoch [1/3], Step [200/414113], Loss: 4.3346, Perplexity: 76.29593
Epoch [1/3], Step [300/414113], Loss: 6.6519, Perplexity: 774.22806
Epoch [1/3], Step [400/414113], Loss: 3.2250, Perplexity: 25.15456
Epoch [1/3], Step [500/414113], Loss: 3.4006, Perplexity: 29.981674
Epoch [1/3], Step [600/414113], Loss: 4.5791, Perplexity: 97.424540
Epoch [1/3], Step [700/414113], Loss: 2.3289, Perplexity: 10.26672
Epoch [1/3], Step [800/414113], Loss: 4.1594, Perplexity: 64.03570
Epoch [1/3], Step [900/414113], Loss: 3.1591, Perplexity: 23.549588
Epoch [1/3], Step [1000/414113], Loss: 2.1412, Perplexity: 8.509665
Epoch [1/3], Step [1100/414113], Loss: 4.1413, Perplexity: 62.883282
Epoch [1/3], Step [1200/414113], Loss: 4.0120, Perplexity: 55.257608
Epoch [1/3], Step [1300/414113], Loss: 3.8427, Perplexity: 46.64904
Epoch [1/3], Step [1400/414113]

Epoch [1/3], Step [11900/414113], Loss: 4.7725, Perplexity: 118.2184
Epoch [1/3], Step [12000/414113], Loss: 2.8794, Perplexity: 17.80334
Epoch [1/3], Step [12100/414113], Loss: 2.1564, Perplexity: 8.640272
Epoch [1/3], Step [12200/414113], Loss: 4.8060, Perplexity: 122.2436
Epoch [1/3], Step [12300/414113], Loss: 3.6914, Perplexity: 40.10197
Epoch [1/3], Step [12400/414113], Loss: 1.7758, Perplexity: 5.9051643
Epoch [1/3], Step [12500/414113], Loss: 2.3350, Perplexity: 10.32935
Epoch [1/3], Step [12600/414113], Loss: 3.3156, Perplexity: 27.53829
Epoch [1/3], Step [12700/414113], Loss: 1.9988, Perplexity: 7.3801384
Epoch [1/3], Step [12800/414113], Loss: 4.6792, Perplexity: 107.6831
Epoch [1/3], Step [12900/414113], Loss: 2.1044, Perplexity: 8.202141
Epoch [1/3], Step [13000/414113], Loss: 3.2356, Perplexity: 25.42123
Epoch [1/3], Step [13100/414113], Loss: 4.0272, Perplexity: 56.102916
Epoch [1/3], Step [13200/414113], Loss: 4.4786, Perplexity: 88.108209
Epoch [1/3], Step [13300/41411

Epoch [1/3], Step [23700/414113], Loss: 2.8627, Perplexity: 17.50943
Epoch [1/3], Step [23800/414113], Loss: 4.8929, Perplexity: 133.3422
Epoch [1/3], Step [23900/414113], Loss: 2.2793, Perplexity: 9.769876
Epoch [1/3], Step [24000/414113], Loss: 2.2804, Perplexity: 9.780571
Epoch [1/3], Step [24100/414113], Loss: 2.1963, Perplexity: 8.991702
Epoch [1/3], Step [24200/414113], Loss: 1.4156, Perplexity: 4.118957
Epoch [1/3], Step [24300/414113], Loss: 2.0429, Perplexity: 7.713102
Epoch [1/3], Step [24400/414113], Loss: 4.6592, Perplexity: 105.5464
Epoch [1/3], Step [24500/414113], Loss: 2.0119, Perplexity: 7.477531
Epoch [1/3], Step [24600/414113], Loss: 3.5662, Perplexity: 35.38278
Epoch [1/3], Step [24700/414113], Loss: 4.2369, Perplexity: 69.19644
Epoch [1/3], Step [24800/414113], Loss: 2.2214, Perplexity: 9.219834
Epoch [1/3], Step [24900/414113], Loss: 6.1331, Perplexity: 460.8695
Epoch [1/3], Step [25000/414113], Loss: 2.6839, Perplexity: 14.641808
Epoch [1/3], Step [25100/414113],

Epoch [1/3], Step [35500/414113], Loss: 4.0291, Perplexity: 56.21107
Epoch [1/3], Step [35600/414113], Loss: 1.9061, Perplexity: 6.726612
Epoch [1/3], Step [35700/414113], Loss: 4.1940, Perplexity: 66.28479
Epoch [1/3], Step [35800/414113], Loss: 4.0062, Perplexity: 54.93849
Epoch [1/3], Step [35900/414113], Loss: 2.0657, Perplexity: 7.890435
Epoch [1/3], Step [36000/414113], Loss: 1.8206, Perplexity: 6.175817
Epoch [1/3], Step [36100/414113], Loss: 4.6938, Perplexity: 109.2637
Epoch [1/3], Step [36200/414113], Loss: 1.7664, Perplexity: 5.8498344
Epoch [1/3], Step [36300/414113], Loss: 2.7890, Perplexity: 16.26480
Epoch [1/3], Step [36400/414113], Loss: 3.7501, Perplexity: 42.525370
Epoch [1/3], Step [36500/414113], Loss: 3.5265, Perplexity: 34.00347
Epoch [1/3], Step [36600/414113], Loss: 2.1976, Perplexity: 9.003319
Epoch [1/3], Step [36700/414113], Loss: 2.3423, Perplexity: 10.40505
Epoch [1/3], Step [36800/414113], Loss: 3.8723, Perplexity: 48.05110
Epoch [1/3], Step [36900/414113]

Epoch [1/3], Step [47300/414113], Loss: 3.4210, Perplexity: 30.60120
Epoch [1/3], Step [47400/414113], Loss: 2.1413, Perplexity: 8.510794
Epoch [1/3], Step [47500/414113], Loss: 1.4539, Perplexity: 4.279961
Epoch [1/3], Step [47600/414113], Loss: 2.0369, Perplexity: 7.6671051
Epoch [1/3], Step [47700/414113], Loss: 3.4676, Perplexity: 32.059769
Epoch [1/3], Step [47800/414113], Loss: 4.3956, Perplexity: 81.094573
Epoch [1/3], Step [47900/414113], Loss: 3.0944, Perplexity: 22.07299
Epoch [1/3], Step [48000/414113], Loss: 4.1755, Perplexity: 65.071406
Epoch [1/3], Step [48100/414113], Loss: 4.0709, Perplexity: 58.60932
Epoch [1/3], Step [48200/414113], Loss: 3.4318, Perplexity: 30.93191
Epoch [1/3], Step [48300/414113], Loss: 2.5234, Perplexity: 12.470674
Epoch [1/3], Step [48400/414113], Loss: 4.0181, Perplexity: 55.59287
Epoch [1/3], Step [48500/414113], Loss: 2.6766, Perplexity: 14.53516
Epoch [1/3], Step [48600/414113], Loss: 3.4671, Perplexity: 32.0427516
Epoch [1/3], Step [48700/41

...

Epoch [1/3], Step [175800/414113], Loss: 4.8674, Perplexity: 129.98446
Epoch [1/3], Step [175900/414113], Loss: 3.5313, Perplexity: 34.169719
Epoch [1/3], Step [176000/414113], Loss: 3.8779, Perplexity: 48.323316
Epoch [1/3], Step [176100/414113], Loss: 2.7630, Perplexity: 15.847968
Epoch [1/3], Step [176200/414113], Loss: 3.9205, Perplexity: 50.42742
Epoch [1/3], Step [176300/414113], Loss: 3.7621, Perplexity: 43.03968
Epoch [1/3], Step [176400/414113], Loss: 2.6250, Perplexity: 13.80471
Epoch [1/3], Step [176500/414113], Loss: 5.0852, Perplexity: 161.6172
Epoch [1/3], Step [176600/414113], Loss: 2.4759, Perplexity: 11.892676
Epoch [1/3], Step [176700/414113], Loss: 3.5951, Perplexity: 36.420939
Epoch [1/3], Step [176800/414113], Loss: 3.7737, Perplexity: 43.54066
Epoch [1/3], Step [176900/414113], Loss: 1.7831, Perplexity: 5.9485278
Epoch [1/3], Step [177000/414113], Loss: 2.2782, Perplexity: 9.759536
Epoch [1/3], Step [177100/414113], Loss: 5.8448, Perplexity: 345.4390
Epoch [1/3], 

Epoch [1/3], Step [187300/414113], Loss: 3.6595, Perplexity: 38.84376
Epoch [1/3], Step [187400/414113], Loss: 5.7828, Perplexity: 324.6691
Epoch [1/3], Step [187500/414113], Loss: 1.2105, Perplexity: 3.3553138
Epoch [1/3], Step [187600/414113], Loss: 5.5035, Perplexity: 245.55516
Epoch [1/3], Step [187700/414113], Loss: 2.0209, Perplexity: 7.545222
Epoch [1/3], Step [187800/414113], Loss: 1.7136, Perplexity: 5.548887
Epoch [1/3], Step [187900/414113], Loss: 3.4745, Perplexity: 32.28246
Epoch [1/3], Step [188000/414113], Loss: 2.3106, Perplexity: 10.080907
Epoch [1/3], Step [188100/414113], Loss: 2.6372, Perplexity: 13.97420
Epoch [1/3], Step [188200/414113], Loss: 2.3168, Perplexity: 10.142707
Epoch [1/3], Step [188300/414113], Loss: 2.7789, Perplexity: 16.101911
Epoch [1/3], Step [188400/414113], Loss: 4.1260, Perplexity: 61.928981
Epoch [1/3], Step [188500/414113], Loss: 6.2185, Perplexity: 501.93454
Epoch [1/3], Step [188600/414113], Loss: 2.5672, Perplexity: 13.029310
Epoch [1/3],

Epoch [1/3], Step [198900/414113], Loss: 1.6778, Perplexity: 5.3535719
Epoch [1/3], Step [199000/414113], Loss: 1.2161, Perplexity: 3.373924
Epoch [1/3], Step [199100/414113], Loss: 3.3777, Perplexity: 29.302603
Epoch [1/3], Step [199200/414113], Loss: 1.6119, Perplexity: 5.012375
Epoch [1/3], Step [199300/414113], Loss: 3.3170, Perplexity: 27.57835
Epoch [1/3], Step [199400/414113], Loss: 1.7417, Perplexity: 5.7071990
Epoch [1/3], Step [199500/414113], Loss: 3.6367, Perplexity: 37.966927
Epoch [1/3], Step [199600/414113], Loss: 3.2231, Perplexity: 25.10631
Epoch [1/3], Step [199700/414113], Loss: 0.9540, Perplexity: 2.5960268
Epoch [1/3], Step [199800/414113], Loss: 4.1443, Perplexity: 63.075593
Epoch [1/3], Step [199900/414113], Loss: 1.8062, Perplexity: 6.0872053
Epoch [1/3], Step [200000/414113], Loss: 3.0787, Perplexity: 21.730057
Epoch [1/3], Step [200100/414113], Loss: 3.1759, Perplexity: 23.948392
Epoch [1/3], Step [200200/414113], Loss: 2.9586, Perplexity: 19.27123
Epoch [1/3]

Epoch [1/3], Step [210400/414113], Loss: 3.2023, Perplexity: 24.588140
Epoch [1/3], Step [210500/414113], Loss: 2.0669, Perplexity: 7.900350
Epoch [1/3], Step [210600/414113], Loss: 2.6420, Perplexity: 14.040757
Epoch [1/3], Step [210700/414113], Loss: 4.7643, Perplexity: 117.25401
Epoch [1/3], Step [210800/414113], Loss: 1.8995, Perplexity: 6.6828722
Epoch [1/3], Step [210900/414113], Loss: 1.3983, Perplexity: 4.0483720
Epoch [1/3], Step [211000/414113], Loss: 3.5059, Perplexity: 33.31164
Epoch [1/3], Step [211100/414113], Loss: 1.5401, Perplexity: 4.665033
Epoch [1/3], Step [211200/414113], Loss: 3.3983, Perplexity: 29.913137
Epoch [1/3], Step [211300/414113], Loss: 2.8015, Perplexity: 16.468999
Epoch [1/3], Step [211400/414113], Loss: 2.4170, Perplexity: 11.21160
Epoch [1/3], Step [211500/414113], Loss: 3.6599, Perplexity: 38.855975
Epoch [1/3], Step [211600/414113], Loss: 2.8961, Perplexity: 18.102804
Epoch [1/3], Step [211700/414113], Loss: 2.5304, Perplexity: 12.55867
Epoch [1/3]

Epoch [1/3], Step [222000/414113], Loss: 3.9707, Perplexity: 53.02023
Epoch [1/3], Step [222100/414113], Loss: 1.8538, Perplexity: 6.3839940
Epoch [1/3], Step [222200/414113], Loss: 1.7234, Perplexity: 5.603699
Epoch [1/3], Step [222300/414113], Loss: 2.0905, Perplexity: 8.08904045
Epoch [1/3], Step [222400/414113], Loss: 2.8504, Perplexity: 17.294356
Epoch [1/3], Step [222500/414113], Loss: 1.1166, Perplexity: 3.054437
Epoch [1/3], Step [222600/414113], Loss: 4.2893, Perplexity: 72.91228
Epoch [1/3], Step [222700/414113], Loss: 3.2053, Perplexity: 24.663526
Epoch [1/3], Step [222800/414113], Loss: 2.4235, Perplexity: 11.285841
Epoch [1/3], Step [222900/414113], Loss: 1.6350, Perplexity: 5.1294437
Epoch [1/3], Step [223000/414113], Loss: 2.0598, Perplexity: 7.8441241
Epoch [1/3], Step [223100/414113], Loss: 1.4603, Perplexity: 4.307043
Epoch [1/3], Step [223200/414113], Loss: 2.3575, Perplexity: 10.564095
Epoch [1/3], Step [223300/414113], Loss: 2.7835, Perplexity: 16.17639
Epoch [1/3]

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.